In [ ]:
#@title mount your Google Drive

import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /path/to/your/project/folder

In [3]:
!git clone https://github.com/veoery/CMU16825_Final_project.git

Cloning into 'CMU16825_Final_project'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (435/435), done.
remote: Compressing objects: 100% (334/334), done.
remote: Total 435 (delta 172), reused 327 (delta 88), pack-reused 0 (from 0)
Receiving objects: 100% (435/435), 2.40 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [ ]:
%cd CMU16825_Final_project

In [ ]:
!pwd
!ls -a

In [ ]:
!uv venv

In [7]:
!source .venv/bin/activate && uv pip install -e .

Using Python 3.12.12 environment at: /usr
Resolved 85 packages in 603ms
Prepared 8 packages in 419ms
Installed 8 packages in 8ms
 + cad-mllm==0.1.0 (from file:///content/gdrive/MyDrive/Fall2025/cmu16825/CMU16825_Final_project)
 + editables==0.5
 + hatchling==1.27.0
 + lightning-utilities==0.15.2
 + pathspec==0.12.1
 + pytorch-lightning==2.5.6
 + torchmetrics==1.8.2
 + trove-classifiers==2025.11.14.15


In [8]:
!source .venv/bin/activate && uv pip install -e ./Michelangelo --no-build-isolation

Using Python 3.12.12 environment at: /usr
Resolved 30 packages in 6.31s
Prepared 1 package in 2.95s
Installed 1 package in 0.91ms
 + michelangelo==0.4.1 (from file:///content/gdrive/MyDrive/Fall2025/cmu16825/CMU16825_Final_project/Michelangelo)


### Inference

In [ ]:
!python scripts/inference.py --prompt "Generate a CAD model of a simple cube." \
    --llm_model_name "Qwen/Qwen3-0.6B"

### Curriculum Training

#### Test with dummy dataset

In [ ]:
# Train from scratch
!python scripts/train_curriculum.py \
    --start_from_stage 1 \
    --create_dummy_data \
    --stage1_epochs 2 \
    --stage2_epochs 2 \
    --stage3_epochs 3 \
    --device cuda

In [ ]:
# Resume from Stage 1 checkpoint and continue with Stages 2 & 3
!python scripts/train_curriculum.py \
    --resume_from_ckpt /path/to/ckpt/stage1_text_model \
    --start_from_stage 2 \
    --create_dummy_data \
    --stage2_epochs 2 \
    --stage3_epochs 3 \
    --device cuda

In [ ]:
# Resume from Stage 3 checkpoint and continue with Stages 3
!python scripts/train_curriculum.py \
    --resume_from_ckpt /path/to/ckpt/stage3_all_model \
    --start_from_stage 3 \
    --create_dummy_data \
    --stage2_epochs 2 \
    --stage3_epochs 3 \
    --device cuda

#### Train with real dataset

Please check all the arguments in `scripts/train_curriculum.py` before training. 

Modify `--batch_size, --gradient_accumulation_steps, --warmup_steps, --max_seq_length, --logging_steps, --save_steps`, `epochs` and `learning rates of 3 stages` and etc to have best performance.

Keep `--llm_model_name "Qwen/Qwen3-8B", --lora_r 32, --lora_alpha 64` fixed.

In [ ]:
# Train with stage 1, 2 and 3 (full curriculum training)
# No need to run this.
!python scripts/train_curriculum.py \
    --use_wandb \
    --omnicad_txt_path ./data/Omni-CAD-subset-complete/txt \
    --omnicad_json_root ./data/Omni-CAD-subset-complete/json \
    --omnicad_img_root ./data/Omni-CAD-subset-complete/img \
    --omnicad_pc_root ./data/Omni-CAD-subset-complete/pointcloud \
    --llm_model_name "Qwen/Qwen3-8B" \
    --start_from_stage 1 \
    --stage1_epochs 1 \
    --stage2_epochs 1 \
    --stage3_epochs 1 \
    --stage1_lr 2e-4 \
    --stage2_lr 2e-4 \
    --stage3_lr 2e-4 \
    --max_seq_length 32768 \
    --batch_size 4 \
    --gradient_accumulation_steps 16 \
    --lora_r 32 \
    --lora_alpha 64 \
    --warmup_steps 100 \
    --logging_steps 10 \
    --save_steps 100 \    
    --device cuda \
    --dtype bfloat16

In [ ]:
# Resume from Stage 1 checkpoint and continue with Stages 2 & 3
!python scripts/train_curriculum.py \
    --use_wandb \
    --omnicad_txt_path ./data/Omni-CAD-subset-complete/txt \
    --omnicad_json_root ./data/Omni-CAD-subset-complete/json \
    --omnicad_img_root ./data/Omni-CAD-subset-complete/img \
    --omnicad_pc_root ./data/Omni-CAD-subset-complete/pointcloud \
    --llm_model_name "Qwen/Qwen3-8B" \
    --resume_from_ckpt /path/to/ckpt/stage1_text_model \
    --start_from_stage 2 \
    --stage2_epochs 1 \
    --stage3_epochs 1 \
    --stage2_lr 2e-4 \
    --stage3_lr 2e-4 \
    --max_seq_length 32768 \
    --batch_size 4 \
    --gradient_accumulation_steps 16 \
    --lora_r 32 \
    --lora_alpha 64 \
    --warmup_steps 100 \
    --logging_steps 10 \
    --save_steps 100 \    
    --device cuda \
    --dtype bfloat16

In [ ]:
# Resume from Stage 1 checkpoint and only run Stage 2
!python scripts/train_curriculum.py \
    --use_wandb \
    --omnicad_txt_path ./data/Omni-CAD-subset-complete/txt \
    --omnicad_json_root ./data/Omni-CAD-subset-complete/json \
    --omnicad_img_root ./data/Omni-CAD-subset-complete/img \
    --omnicad_pc_root ./data/Omni-CAD-subset-complete/pointcloud \
    --llm_model_name "Qwen/Qwen3-8B" \
    --resume_from_ckpt /path/to/ckpt/stage1_text_model \
    --start_from_stage 2 \
    --stage2_epochs 1 \
    --stage3_epochs 0 \
    --stage2_lr 2e-4 \
    --stage3_lr 2e-4 \
    --max_seq_length 32768 \
    --batch_size 4 \
    --gradient_accumulation_steps 16 \
    --lora_r 32 \
    --lora_alpha 64 \
    --warmup_steps 100 \
    --logging_steps 10 \
    --save_steps 100 \    
    --device cuda \
    --dtype bfloat16